<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebooks" data-toc-modified-id="Notebooks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notebooks</a></span><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Mark-false-positive-notebooks-and-notebooks-with-broken-format" data-toc-modified-id="Mark-false-positive-notebooks-and-notebooks-with-broken-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Mark false-positive notebooks and notebooks with broken format</a></span></li><li><span><a href="#Mark-empty-notebooks" data-toc-modified-id="Mark-empty-notebooks-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Mark empty notebooks</a></span></li><li><span><a href="#Mark-fork-duplicates" data-toc-modified-id="Mark-fork-duplicates-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Mark fork duplicates</a></span></li><li><span><a href="#Mark-duplicates" data-toc-modified-id="Mark-duplicates-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Mark duplicates</a></span></li><li><span><a href="#Mark-restricted-toy" data-toc-modified-id="Mark-restricted-toy-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Mark restricted toy</a></span></li><li><span><a href="#Mark-toy-examples" data-toc-modified-id="Mark-toy-examples-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Mark toy examples</a></span></li></ul></li></ul></div>

# Notebooks

Analyze notebooks: programming languages, python version, number of cells by notebookk, and notebook names.

In [1]:
import sys
sys.path.insert(0, '../archaeology')

from string import ascii_letters, digits

import tqdm

import numpy as np
import pandas as pd

from db import connect

import analysis_helpers, importlib
importlib.reload(analysis_helpers)
from analysis_helpers import load_vars, var, relative_var

def elite(column):
    column = column.dropna()
    column = column[column > 0]
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    iqr = q3 - q1
    return q3 + 1.5*iqr

%matplotlib inline

## Load

In [2]:
with connect() as session:
    print("Query repositories")
    repositories = pd.read_sql_table("repositories", session.connection())

Query repositories


In [3]:
with connect() as session:
    print("Query Notebooks")
    notebooks = pd.read_sql_table("notebooks", session.connection())

Query Notebooks


In [4]:
with connect() as session:
    print("Query repository updates")
    repository_updates = pd.read_sql_table("repository_updates", session.connection())

Query repository updates


Join notebooks and repository updates

In [5]:
notebooks = notebooks.join(repository_updates.set_index("repository_id"), on="repository_id", rsuffix='_ru')

Set skip = 0

In [6]:
notebooks.loc[:, "skip"] = 0

## Mark notebooks from removed repositories

In [7]:
mask = notebooks["history_commit_date"].isna()
SKIP = 2048
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

175199

## Mark false-positive notebooks and notebooks with broken format

In [8]:
mask = ~(
    ~((notebooks["kernel"] == "no-kernel")
     &(notebooks["nbformat"] == "0"))
    &(notebooks["total_cells"] != 0)
)
SKIP = 1024
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

14716

## Mark empty notebooks

In [9]:
mask = notebooks["total_cells"] == notebooks["empty_cells"]
SKIP = 512
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

26395

## Mark fork duplicates

First, prioritize non-fork notebooks and notebooks from the same repository

In [10]:
notebooks.sort_values(["parent_repository", "repository_id", "id"], na_position="first", inplace=True)

Mark the notebooks

In [11]:
mask = notebooks.duplicated(subset=['name', 'sha1_source'])
SKIP = 256
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

137181

## Mark duplicates

In [12]:
mask = notebooks.duplicated(subset=['sha1_source'])
SKIP = 128
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

290312

## Mark restricted toy

In [13]:
mask = (
    (notebooks['homework_count'] != 0)
    | ((notebooks['course_count'] != 0) & (
        (notebooks['assignment_count'] != 0)
        | (notebooks['lesson_count'] != 0)
        | (notebooks['exercise_count'] != 0)
    ))
)
SKIP = 64
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

106619

## Mark toy examples

In [14]:
mask = (
    (notebooks['homework_count'] != 0)
    | (notebooks['course_count'] != 0)
    | (notebooks['lesson_count'] != 0)
    | (notebooks['assignment_count'] != 0)
    | (notebooks['exercise_count'] != 0)
)
SKIP = 32
notebooks.loc[mask, 'skip'] = notebooks[mask]['skip'] | SKIP
len(notebooks[np.bitwise_and(notebooks["skip"], SKIP) == SKIP])

357590

## Add stars, forks and metric to notebooks

In [15]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set starforks = 2.0 / (1.0/ru.stargazers + 1.0/ru.forks),
    stargazers = ru.stargazers,
    forks = ru.forks
    from repository_updates ru
    where ru.repository_id = notebooks.repository_id
    and ru.stargazers != 0
    and ru.forks != 0
    """)
    session.commit()

CPU times: user 10.4 ms, sys: 304 µs, total: 10.7 ms
Wall time: 5.15 s


In [16]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set starforks = 0,
    stargazers = ru.stargazers,
    forks = ru.forks
    from repository_updates ru
    where ru.repository_id = notebooks.repository_id
    and (ru.stargazers = 0 or ru.forks = 0)
    """)
    session.commit()

CPU times: user 9.87 ms, sys: 277 µs, total: 10.1 ms
Wall time: 14.9 s


In [17]:
notebooks.loc[:, 'starforks'] = 2.0 / ((1.0 / notebooks["stargazers"]) + (1.0 / notebooks["forks"]))

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


## Distinguish elite groups

Get existing non duplicated valid notebooks (endv_notebooks)

In [18]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set sfgroup = 0;
    """)
    session.commit()

CPU times: user 274 ms, sys: 709 ms, total: 983 ms
Wall time: 15 s


In [19]:
mask = (
    np.bitwise_and(notebooks["skip"], 2048 + 1024 + 512 + 128) == 0
)
endv_notebooks = notebooks[mask]


In [20]:
elite_stars = elite(endv_notebooks['stargazers'])
print("Elite stars:", var("s_a0_stars", elite_stars))

Elite stars: 21.0


In [21]:
column = endv_notebooks['stargazers']
column = column[column > elite_stars]
super_stars = elite(column)
print("Super elite stars:", var("ss_a0_stars", super_stars))

Super elite stars: 311.0


In [22]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set sfgroup = sfgroup | 1
    where stargazers > {}
    """.format(elite_stars))
    session.commit()

CPU times: user 243 ms, sys: 678 ms, total: 921 ms
Wall time: 3.12 s


In [23]:
elite_forks = elite(endv_notebooks['forks'])
print("Elite forks:", var("s_a0_forks", elite_forks))

Elite forks: 21.0


In [24]:
column = endv_notebooks['forks']
column = column[column > elite_forks]
super_forks = elite(column)
print("Super elite forks:", var("ss_a0_forks", super_forks))

Super elite forks: 221.5


In [25]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set sfgroup = sfgroup | 2
    where forks > {}
    """.format(elite_forks))
    session.commit()

CPU times: user 218 ms, sys: 633 ms, total: 851 ms
Wall time: 1.25 s


In [26]:
elite_sf = elite(endv_notebooks['starforks'])
print("Elite starforks:", var("s_a0_starforks", elite_sf))

Elite starforks: 33.331395348837205


In [27]:
column = endv_notebooks['starforks']
column = column[column > elite_sf]
super_sf = elite(column)
print("Super elite starforks:", var("ss_a0_starforks", super_sf))

Super elite starforks: 360.9189931689931


In [28]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set sfgroup = sfgroup | 4
    where starforks > {}
    """.format(elite_sf))
    session.commit()

CPU times: user 183 ms, sys: 693 ms, total: 875 ms
Wall time: 1.06 s


## Update Skips

In [29]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks
    set skip = 0;
    """)
    session.commit()

CPU times: user 2.7 ms, sys: 7.88 ms, total: 10.6 ms
Wall time: 15.1 s


In [30]:
with_skip = notebooks[notebooks["skip"] != 0]

with connect() as session:
    for id_, skip in tqdm.tqdm(zip(with_skip['id'], with_skip['skip']), total=len(with_skip)):
        #print(id_)
        session.execute("UPDATE notebooks SET skip = {} WHERE id = {};".format(skip, id_))
    session.commit()

100%|██████████| 649587/649587 [02:13<00:00, 4860.36it/s]


In [31]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebook_features AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()
    

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 14.4 s


In [32]:
%%time
with connect() as session:  
    session.execute("""
    UPDATE notebook_modules AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()
    

CPU times: user 9.89 ms, sys: 250 µs, total: 10.1 ms
Wall time: 14.9 s


In [33]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebook_asts AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 10.5 ms, sys: 4.69 ms, total: 15.2 ms
Wall time: 24.5 s


In [34]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebook_names AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 5.12 ms, sys: 7.67 ms, total: 12.8 ms
Wall time: 1min 5s


In [35]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebook_markdowns AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 11.3 ms, sys: 3.37 ms, total: 14.6 ms
Wall time: 32.1 s


In [36]:
%%time
with connect() as session:
    session.execute("""
    UPDATE executions AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 18.1 ms, sys: 1.37 ms, total: 19.4 ms
Wall time: 5min 6s


In [37]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks_bool_aggregates AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 16.5 ms, sys: 0 ns, total: 16.5 ms
Wall time: 53.7 s


In [38]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks_meta_aggregates AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 18.7 ms, sys: 1.2 ms, total: 19.9 ms
Wall time: 3min 10s


In [39]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks_fullexecbool_aggregates AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 10.6 ms, sys: 341 µs, total: 10.9 ms
Wall time: 15.9 s


In [40]:
%%time
with connect() as session:
    session.execute("""
    UPDATE notebooks_fullexecmeta_aggregates AS u
    SET skip = n.skip
    FROM notebooks AS n
    WHERE u.notebook_id = n.id
    """)
    session.commit()

CPU times: user 12.4 ms, sys: 595 µs, total: 13 ms
Wall time: 2min 6s


This notebook prepared the skip attributes of all tables to be used in other analyses